# Geometrical relaxations from pretrained m3gnet model

In [1]:
import ML_library as MLL
import matgl
import warnings

from pymatgen.io.vasp.inputs import Poscar
from __future__              import annotations
from pymatgen.core           import Structure
from matgl.ext.ase           import M3GNetCalculator, Relaxer
from pymatgen.io.ase         import AseAtomsAdaptor

# To suppress warnings for clearer output
warnings.simplefilter('ignore')

In [2]:
# Define paths to pretrained model and structure to be relaxed

# Materials Project pretrained model as default
model_load_path = 'finetuned_model'
model_load_path = 'M3GNet-MP-2021.2.8-PES' if model_load_path is None else model_load_path

path_to_POSCAR  = 'examples'

# Structure Relaxation

To perform structure relaxation, we use the Relaxer class.

In [3]:
# Load the structure to be relaxed
atoms_ini = Structure.from_file(f'{path_to_POSCAR}/POSCAR')

In [4]:
# Clear caché of previous model
matgl.clear_cache()

# Load the default pre-trained model
pot = matgl.load_model(model_load_path)
relaxer = Relaxer(potential=pot)

In [5]:
# Relax the structure
relax_atoms_ini = relaxer.relax(atoms_ini, verbose=True)
atoms = relax_atoms_ini['final_structure']

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 14:14:57      -14.875906*      66.3234
FIRE:    1 14:14:57        0.904017*     192.7172
FIRE:    2 14:14:57      -14.878324*      66.4468
FIRE:    3 14:14:57      -15.069678*      42.9724
FIRE:    4 14:14:57      -15.523190*      30.8008
FIRE:    5 14:14:58      -15.918114*       7.4548
FIRE:    6 14:14:58      -15.697193*      28.5399
FIRE:    7 14:14:58      -15.748351*      25.1294
FIRE:    8 14:14:58      -15.826632*      19.0197
FIRE:    9 14:14:58      -15.897688*      11.3148
FIRE:   10 14:14:58      -15.933854*       3.1205
FIRE:   11 14:14:58      -15.924374*       6.4485
FIRE:   12 14:14:58      -15.924948*       6.3216
FIRE:   13 14:14:59      -15.926044*       6.0700
FIRE:   14 14:14:59      -15.927551*       5.6988
FIRE:   15 14:14:59      -15.929352*       5.2151
FIRE:   16 14:14:59      -15.931279*       4.6285
FIRE:   17 14:14:59      -15.933165*       3.9510
FI

In [6]:
# Save the relaxed structure as a POSCAR file
poscar_relaxed = Poscar(atoms)
poscar_relaxed.write_file(f'{path_to_POSCAR}/CONTCAR')

In [7]:
# Or as implemented
poscar_relaxed = MLL.structural_relaxation(path_to_POSCAR, model_load_path)

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 14:15:26      -14.875906*      66.3234
FIRE:    1 14:15:26        0.904014*     192.7172
FIRE:    2 14:15:26      -14.878327*      66.4467
FIRE:    3 14:15:26      -15.069677*      42.9724
FIRE:    4 14:15:26      -15.523190*      30.8007
FIRE:    5 14:15:26      -15.918115*       7.4549
FIRE:    6 14:15:27      -15.697195*      28.5398
FIRE:    7 14:15:27      -15.748351*      25.1294
FIRE:    8 14:15:27      -15.826634*      19.0197
FIRE:    9 14:15:27      -15.897689*      11.3147
FIRE:   10 14:15:27      -15.933858*       3.1205
FIRE:   11 14:15:27      -15.924375*       6.4485
FIRE:   12 14:15:27      -15.924947*       6.3216
FIRE:   13 14:15:28      -15.926041*       6.0699
FIRE:   14 14:15:28      -15.927552*       5.6988
FIRE:   15 14:15:28      -15.929351*       5.2151
FIRE:   16 14:15:28      -15.931280*       4.6285
FIRE:   17 14:15:28      -15.933162*       3.9511
FI

# Single-shot energy calculation

Perform a single-shot calculation for final structure using M3GNetCalculator.

In [8]:
# Define the M3GNet calculator
calc = M3GNetCalculator(pot)

# Load atoms adapter and adapt structure
ase_adaptor = AseAtomsAdaptor()
adapted_atoms = ase_adaptor.get_atoms(atoms)

# Calculate potential energy
adapted_atoms.set_calculator(calc)
print(f'The calculated potential energy is {float(adapted_atoms.get_potential_energy()):.3f} eV/supercell')

The calculated potential energy is -16.117 eV/supercell


In [9]:
# Or as implemented
ssc_energy = MLL.single_shot_energy_calculations(f'{path_to_POSCAR}/CONTCAR', model_load_path)
print(f'The calculated potential energy is {ssc_energy:.3f} eV/supercell')

The calculated potential energy is -16.119 eV/supercell


In [10]:
# Try on 3x3x3 supercell
ssc_energy = MLL.single_shot_energy_calculations(f'{path_to_POSCAR}/333_supercell/POSCAR', model_load_path)
print(f'The calculated potential energy is {ssc_energy:.3f} eV/supercell')

FileNotFoundError: [Errno 2] No such file or directory: 'examples/333_supercell/POSCAR'